# Test the transformation on data from the actual problem.

We developed the new equality constraint

$\sum V_{\Delta} \cdot P = c ||P||_1$

Our goal is to take elements of a space

$V_{\Delta}''$ where $l \leq (V_{\Delta})_i \leq u$ for all $i$

and transform into a vector satisfying the condition above.

(Remember we go from $n-1$ dimensional space to $n$ dimensional space.)

In [29]:
# load libraries
import numpy as np
import pandas as pd
import scipy

In [8]:
# load and clean data
with open("../project_dir.txt") as f:
    project_path = f.read().strip() + "{}"
    
print(project_path)
# vaccination data and population data from sifat
vacc_df = pd.read_csv(project_path.format("data/VA_zipcodes_cleaned/ZC_immunization_sifat.csv"))
# drop 0 population entries, they won't affect the simulation
vacc_df = vacc_df[vacc_df['population'] > 0].reset_index(drop=True)
vacc_df.rename({'population':'pop'},axis=1,inplace=True)
# load distance matrix computed from nominatim and geopy distance function
dist_df = pd.read_csv(project_path.format("data/VA_zipcodes_cleaned/ZC_distance_sifat_nom_geopy.csv"))
# need to replace 0's in distance matrix to avoid divide by zero in gravity formula
default_dist = 0.5
dist_df.loc[dist_df[np.isclose(dist_df['distKM'],0)].index,'distKM']=default_dist
# convert to matrix
dist_mat = dist_df.pivot(index='zipcode1',columns='zipcode2',values='distKM')
dist_mat = dist_mat.replace(np.nan,0)
# align matrix
dist_mat = dist_mat.loc[vacc_df['zipcode'],vacc_df['zipcode']]

top_5 = vacc_df.sort_values(by='pop',ascending=False).head(5)
I = np.zeros(len(vacc_df.index))
np.put(I,top_5.index,1)


/home/nicholasw/Documents/sync/UVA files/Summer 2022 (C4GC with BII)/measles_metapop/{}


In [72]:
constraint_bnd = 0.05
V_0 = np.array((vacc_df['pop'] - vacc_df['nVaccCount'])/(vacc_df['pop']))
dim = len(V_0)
print(dim)
eq_constraint_bnd = constraint_bnd*sum(vacc_df['pop'])

705


# check the forward transformation again

In [97]:
def generate_v_delta_candidate(V_0):
    V_delta = np.zeros(len(V_0))
    for i,value in enumerate(V_0):
        V_delta[i] = np.random.random()*value
    return V_delta

In [98]:
# doing the change of variable
test_point = generate_v_delta_candidate(V_0)*np.array(vacc_df['pop'])

In [100]:
sum(test_point)

3433192.5305393278

In [101]:
def generate_number_summing_to(sum_target,dim,n_points):
    # better way of generating random numbers?
    points = []
    while len(points) < n_points:
        x = [np.random.random() for i in range(dim)]
        x.insert(0,0)
        x.append(1)
        x.sort()
        points.append(np.diff(x)*sum_target)
    points = np.float128(points)
    return points

In [ ]:
def generate_number_sum_(sum_target,dim,n_points):
    # better way of generating random numbers?
    points = []
    while len(points) < n_points:
        x = [np.random.random() for i in range(dim)]
        x.insert(0,0)
        x.append(1)
        x.sort()
        points.append(np.diff(x)*sum_target)
    points = np.float128(points)
    return points

In [105]:
generate_number_summing_to(eq_constraint_bnd,dim-1,1)[0]/vacc_df['pop']

0      0.198522
1      0.075929
2      1.292146
3      0.025020
4      0.434745
         ...   
700    0.505635
701    0.044569
702    0.003549
703    0.044789
704    0.002500
Name: pop, Length: 705, dtype: float128

In [73]:
# now let's generate some candidate vectors in our new space

# todo: what to choose for lower and upper bounds? idk
l = 10
u = 110
n_points = 100
points = np.array([np.array([np.random.random()*(u-l)+l for i in range(dim-1)]) for i in range(n_points)])
len(points[0])

704

In [74]:
# maybe we can make it better by posing it as a linear system
# and hoping that the np solver does something smart
from scipy.optimize import root
def inv_tr_lin_solve_eq(point,l,u,c):
    dim = len(point)
    coeff_matrix = np.eye(dim+1)
    # add bottom row of ones
    #coeff_matrix[dim-1,:] = np.ones(dim)
    # add coefficients to off-diagonal
    for i in range(0,dim):
        coeff_matrix[i,i+1] = (l-point[i])/(u-l)
    #print(coeff_matrix)
    b = np.zeros(dim+1)
    b[dim] = c
    #print(b)
    L = np.tril(np.ones((dim+1,dim+1)))
    x = scipy.linalg.solve(coeff_matrix@L,b)
    return x
    #def objective(x):
    #    return (coeff_matrix@L)@x-b
    #return root(objective,np.ones(dim))

In [75]:
inv_tr_lin_solve_eq(points[0],l,u,c=eq_constraint_bnd)

array([-3.66485408e-11,  1.89968837e-11, -2.31837744e-11, -2.41233671e-11,
        2.82170981e-12,  4.42379930e-11, -1.18088399e-11,  2.76445502e-11,
       -1.36512641e-11, -5.12624569e-13,  1.38152309e-11, -6.81853636e-13,
       -3.18808740e-12, -2.63615635e-11,  2.77707275e-11, -4.28262179e-12,
       -1.64853419e-12, -2.31080100e-12, -2.30028792e-11, -2.30014900e-11,
        5.44288895e-11,  1.86128052e-11,  1.76555037e-11, -1.07435328e-11,
       -3.12533015e-11,  6.04999245e-11, -1.08679494e-10,  1.58843858e-11,
       -8.88409844e-13,  1.71464260e-11, -1.64119030e-12,  3.01569048e-11,
        3.76750549e-12,  7.43063678e-12,  2.07070217e-11,  2.31792849e-11,
       -6.49146507e-11, -4.34481578e-12, -1.03348670e-11, -5.79010471e-12,
        1.08963721e-11, -6.78104386e-12, -2.60896972e-11, -8.19723397e-12,
        7.22157572e-13,  7.25070751e-11, -4.09901064e-11,  1.19340024e-11,
       -2.78180338e-12,  1.40065680e-11, -8.15889698e-12, -7.48819203e-12,
       -4.21879171e-11,  